# Setup

In [17]:
import talib as ta
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pickle
import tiingo
import financedatabase as fd
import datetime
import financetoolkit
from utils.api_keys import API_KEYS
import zipfile
import requests

# Tiingo

We use the [supported tickers by Tiingo](https://www.tiingo.com/documentation/end-of-day#:~:text=supported_tickers.zip) to select ETFs that are still listed on the following: 
*   Nasdaq
*   NYSE
*   BATS

The ETFs must be traded prior to 2018 and still be active.

This selection of ETFs resulted in a list of 1789 ETFs.

In [9]:
# Download the Tiingo supported tickers file from the URL
tiingo_zip_url = 'https://apimedia.tiingo.com/docs/tiingo/daily/supported_tickers.zip'
#unzip the file
tiingo_zip_path = 'supported_tickers.zip'
# Download the zip file
response = requests.get(tiingo_zip_url)
with open(tiingo_zip_path, 'wb') as f:
    f.write(response.content)
# Unzip the file
with zipfile.ZipFile(tiingo_zip_path, 'r') as zip_ref:
    zip_ref.extractall('.')
 
# Read the CSV file
supported_tickers_df = pd.read_csv(tiingo_zip_path)
# Filter the DataFrame to include only ETFs
etfs_df = supported_tickers_df[supported_tickers_df['assetType'] == 'ETF']
# Filter the DataFrame to include only the tickers that are not delisted
etfs_df = etfs_df[etfs_df['endDate'] >= '2025-04-23']
# Filter the DataFrame to include only the tickers traded prior to 2018
etfs_df = etfs_df[etfs_df['startDate'] <= '2018-01-01']
# Filter the DataFrame to include only the tickers traded on NYSE, NASDAQ or BATS
etfs_df = etfs_df[etfs_df['exchange'].isin(['NYSE', 'NYSE ARCA', 'NYSE MKT', 'NASDAQ', 'BATS'])]
# return the tickers
etf_tickers = etfs_df['ticker'].tolist()
# display the first 10 tickers
print("First 10 ETF tickers:")
print(etf_tickers[:10])


First 10 ETF tickers:
['AADR', 'AAVM', 'AAXJ', 'ABFL', 'ABLG', 'ACP', 'ACSI', 'ACV', 'ACWI', 'ACWV']


In [16]:
len(etf_tickers)

1789

# Enriching with 

In [22]:
#Initialize the financedatabase ETFs Data
etfs = fd.ETFs()
etfs_data = etfs.data
etfs_data

,name,currency,summary,category_group,category,family,exchange
symbol,,,,,,,
^ACWI,ISHARES TRUST,USD,The iShares MSCI ACWI ETF seeks to track the i...,Financials,Developed Markets,BlackRock Asset Management,NIM
^ADFI-IV,NFIELD DYNAMIC FIXED INCOME ETF,USD,The NFIELD DYNAMIC FIXED INCOME ETF (ADFI) is ...,Fixed Income,Corporate Bonds,NaN,ASE
^ADRE,INVESCO ACTIVELY M,USD,The Invesco Active REIT ETF is an actively man...,Real Estate,REITs,Invesco Investment Management,NIM
^ARB-EU,ALTSHARES MERGER ARBITRAGE ETF,USD,The ALTSHARES MERGER ARBITRAGE ETF seeks capit...,Alternatives,NaN,AltShares,ASE
^ARB-IV,ALTSHARES MERGER ARBITRAGE ETF,USD,The ALTSHARES MERGER ARBITRAGE ETF seeks capit...,Alternatives,NaN,AltShares,ASE
...,...,...,...,...,...,...,...
VGFPF,Vanguard Funds Public Limited Company - Vangua...,NaN,The Vanguard S&P 500 UCITS ETF (USD) Accumulat...,Equities,NaN,NaN,NaN
VFDEF,Vanguard Funds Public Limited Company - Vangua...,NaN,VFDEF is an exchange-traded fund (ETF) that ai...,Equities,NaN,NaN,NaN
WSDMF,WisdomTree Issuer ICAV - WisdomTree Europe Equ...,NaN,The WisdomTree Issuer ICAV - WisdomTree Europe...,Equities,NaN,NaN,NaN


In [30]:

# Build dictionnary for sector, industry and category
etf_sector_info = {}

# For each selected Ticker, get the sector, industry and category
enriched_etfs = []
for ticker in etf_tickers:
    # Look for additional information in financedatabase
    # we may need to handle suffixes (ex: adding .US)
    if ticker in etfs_data.index:
        etf_info = etfs_data.loc[ticker]
        enriched_etfs.append({
            'ticker': ticker,
            'name': etf_info['name'] if 'name' in etf_info else 'N/A',
            'sector': etf_info['category_group'] if 'category_group' in etf_info else 'N/A',
            'industry': etf_info['category'] if 'category' in etf_info else 'N/A'
        })
    else:
        # Try other formats
        alt_ticker = ticker + '.US'
        if alt_ticker in etfs_data:
            etf_info = etfs_data.loc[alt_ticker]
            enriched_etfs.append({
                'ticker': ticker,
                'name': etf_info.get('name', 'N/A'),
                'sector': etf_info.get('category_group', 'N/A'),
                'industry': etf_info['category'] if 'category' in etf_info else 'N/A'
            })
        else:
            # Ticker not found in financedatabase, add a placeholder
            enriched_etfs.append({
                'ticker': ticker,
                'name': 'N/A',
                'sector': 'N/A',
                'industry': 'N/A',
                'category': 'N/A'
            })

# Build a DataFrame from the enriched ETFs list
enriched_etfs_df = pd.DataFrame(enriched_etfs)

# Filter the DataFrame to include only ETFs with sector, industry or category containing 'Technology'
tech_etfs_df = enriched_etfs_df[
                                (enriched_etfs_df['sector'].str.contains('Technology', na=False)) |
                                (enriched_etfs_df['industry'].str.contains('Technology', na=False))
                                ].reset_index(drop=True)                           

# Afficher le résultat
print(f"Tech ETFs found: {len(tech_etfs_df)}")
tech_etfs_df.head()

Tech ETFs found: 75


,ticker,name,sector,industry,category
0,ARKK,ARK Innovation ETF,Information Technology,Factors,NaN
1,ARKQ,ARK Autonomous Technology & Robotics ETF,Industrials,Technology,NaN
2,ARKW,ARK Next Generation Internet ETF,Information Technology,Technology,NaN
3,CIBR,First Trust NASDAQ Cybersecurity ETF,Information Technology,Technology,NaN
4,CQQQ,Invesco China Technology ETF,Information Technology,Emerging Markets,NaN
